logica: miro una fecha y un bidder, miro sus competidores, busco dias similares y comportamiento de sus competidores. Despues miro M muestra boostratps, como escenarios y veo el equilibrio, de como juegan sus competidores. Después miro, en cada una de esas M muestras bootstratps para ese dia hallo la derivada, despues hago el promedio sobre todas las muestras bootstratps. Depues hago la suma sobre todas las horas y eso va a ser el phi, con los otros terminos

In [1]:
#librerias
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.stats import norm
from tqdm.auto import tqdm
from tqdm_joblib import tqdm_joblib
import os

In [2]:
#cargar archivos
df_final = pd.read_csv("../datos/procesado/df_final.csv")
df_transado =pd.read_csv("../datos/procesado/df_despacho_agg.csv")

In [3]:
df_transado = df_transado.rename(columns={'daily_eq_demand': 'demanda'})
df_final= df_final[df_final['cantidad']!=0]


In [4]:
#params
M=1000
gamma=10

In [5]:
# ----------------------------------------------------
# FUNCIONES AUXILIARES
# ----------------------------------------------------

def get_cluster_it(df, fecha, firma):
    """Devuelve el cluster correspondiente a una firma i en una FechaHora t."""
    row = df.loc[
        (df['FechaHora'] == fecha) & (df['CodigoPlanta'] == firma),
        'cluster'
    ]
    return row.iloc[0] if not row.empty else np.nan


def get_competitors(df, fecha, firma):
    """Devuelve los competidores (otros CodigoPlanta) presentes en la misma FechaHora."""
    df_day = df[df['FechaHora'] == fecha]
    competitors = df_day.loc[df_day['CodigoPlanta'] != firma, 'CodigoPlanta'].unique().tolist()
    return competitors


def get_similar_days_by_cluster(df, fecha, firma, max_obs=20):

    cluster_it = get_cluster_it(df, fecha, firma)
    competitors = get_competitors(df, fecha, firma)

    # cantidad de la firma en esa fecha
    cantidad_i = df.loc[(df['CodigoPlanta'] == firma) & (df['FechaHora'] == fecha), 'cantidad'].item()
    
    
    similar_obs = []

    for comp in competitors:

        df_comp_similar = df[
            (df['CodigoPlanta'] == comp) &
            (df['cluster'] == cluster_it)
        ].copy()

        # calcular distancia en cantidad a la firma
        df_comp_similar['dist_cantidad'] = (
            df_comp_similar['cantidad'] - cantidad_i
        ).abs()

        # ordenar por cercanía y tomar máximo max_obs
        df_comp_similar = (
            df_comp_similar
            .sort_values('dist_cantidad')
            .head(max_obs)
        )

        df_comp_similar['competidor_de'] = firma
        df_comp_similar['fecha_base'] = fecha

        similar_obs.append(df_comp_similar)

    return pd.concat(similar_obs, ignore_index=True) if similar_obs else pd.DataFrame()


def bootstrap_by_planta(df, M, seed=None):
    """Genera M muestras bootstrap independientes seleccionando 1 observación por planta."""
    if seed is not None:
        np.random.seed(seed)

    plantas = df['CodigoPlanta'].unique()
    bootstrap_samples = []

    for m in range(M):
        muestras = []
        for p in plantas:
            df_p = df[df['CodigoPlanta'] == p]
            if len(df_p) == 0:
                continue
            muestra = df_p.sample(1, replace=True)
            muestra['bootstrap_id'] = m + 1
            muestras.append(muestra)
        sample_df = pd.concat(muestras).reset_index(drop=True)
        bootstrap_samples.append(sample_df)

    return bootstrap_samples


def compute_equilibrium(df_offers, df_transado_date):
    """
    Encuentra el precio y cantidad de equilibrio (p*, q*) para un conjunto de ofertas.
    df_transado_date debe contener la demanda para la FechaHora actual.
    """
    df_transado_date['FechaHora'] = pd.to_datetime(df_transado_date['FechaHora'], errors='coerce')
    df_offers['fecha_base'] = pd.to_datetime(df_offers['fecha_base'], errors='coerce')

    fecha = df_offers['fecha_base'].iloc[0]
   

    demanda_row = df_transado_date.loc[df_transado_date['FechaHora'] == fecha, 'demanda']
    
    
    if demanda_row.empty:
        return np.nan, np.nan

    demand = demanda_row.iloc[0]
    df_sorted = df_offers.sort_values('precio').copy()
    df_sorted['acum'] = df_sorted['cantidad_pos'].cumsum()

    clearing_offers = df_sorted[df_sorted['acum'] >= demand]
    if clearing_offers.empty:
        return np.nan, demand

    p_star = clearing_offers.iloc[0]['precio']
    q_star = demand
    
    return p_star, q_star

In [6]:
# ----------------------------------------------------
# KERNELS
# ----------------------------------------------------
# --- 1. Definiciones del Kernel ---

def gaussian_kernel(u):
    """Kernel Gaussiano estándar (PDF de N(0, 1))."""
    return norm.pdf(u)

def gaussian_kernel_prime(u):
    """Derivada del Kernel Gaussiano: κ'(u) = -u * κ(u)."""
    return -u * gaussian_kernel(u)   #ya es como si tuviera el negativo, u lo reescribo como  pht-pkt por justificacion

# --- 2. Estimación de la Demanda Residual (RD(p)) ---

# Asumo que tienes una función para obtener pos_it, o que se añade como argumento

def kernel_expectation(df, p_ht, D, gamma, df_firma):
    """
    Calcula la Demanda Residual Neta:
       RD(p) = D - sum_k g_k * K((b_k - p)/gamma)
    para la firma i.
    """
    #cantidad pos es cantidad descontada por contrato bilateral
    if df.empty:
        return D   # si no hay rivales, demanda residual = demanda total

   
    gammai=gamma* df_firma['gamma_thumb'].values[0]
   
    bit=df_firma['precio'].values[0]
    # Argumento del kernel
    u_others = (df["precio"] - bit) / gammai

    # Kernel gaussiano
    weights_others = gaussian_kernel(u_others)

    # --- AQUÍ LA CORRECCIÓN CLAVE ---
     # opción 1: binaria
    weight = (df["precio"] < p_ht).astype(float)
    # opción 2: continua usando sigmoid
    beta=5
    #weight = 1 / (1 + np.exp(-beta * (df["precio"] - p_ht)))

    # SUMA ponderada
    
    # Es una SUMA ponderada, NO un promedio
    S_minus_i = (df['cantidad_pos'] * weights_others* weight).sum()
    
    
    # Demanda residual bruta
    RD_p = (D - S_minus_i).values[0]
    q_min = df_firma['cantidad_pos'].values[0]
    resultado = min(RD_p, q_min)
    
    
    return resultado


# --- 3. Estimación de la Derivada de la Demanda Residual (RD'(p)) ---

def kernel_derivative(df, p_ht, gamma, df_firma):
    """
    Calcula la derivada de la Demanda Residual:
        RD'(p) = sum_{k ≠ i} g_k * (1/gamma) * K'((b_k - p)/gamma)
    """
    if df.empty:
        return 0.0
    gammai=gamma* df_firma['gamma_thumb'].values[0]
    # Coincidir EXACTAMENTE con tu fórmula: (b_k - p_ht)/gamma
    u_others = (df["precio"] - p_ht) / gammai

    # K'(u) = -u K(u)
    weights_prime = gaussian_kernel_prime(u_others)

    # SUMA ponderada (no promedio)
    dQ = (df["cantidad"] * weights_prime).sum() / gammai
    
    return dQ




def kernel_derivative_weighted(df, p_ht, gamma, df_firma):
    """
    Calcula la derivada de la Demanda Residual ponderando más los bids que no entran
    RD'(p) = sum_{k ≠ i} g_k * (1/gamma) * K'((b_k - p)/gamma) * w(b_k, p_ht)
    """
    if df.empty:
        return 0.0
    bit=df_firma['precio'].values[0]
    gammai=gamma*df_firma['gamma_thumb'].values[0]
    # Vector u = (b_k - p_h)/gamma
    u_others = (df["precio"] - p_ht) / gammai

    # derivada del kernel normal: K'(u) = -u K(u)
    weights_prime = gaussian_kernel_prime(u_others)

    # Ponderación: más peso si b_k > p_h
    # opción 1: binaria
    weight = (df["precio"] > p_ht).astype(float)
    # opción 2: continua usando sigmoid
    beta=5
    #weight = 1 / (1 + np.exp(-beta * (df["precio"] - p_ht)))

    # SUMA ponderada
    dQ = (df["cantidad"] * weights_prime * weight).sum() / gammai
    
    return dQ



In [7]:
# ----------------------------------------------------
# PROMEDIO SOBRE BOOTSTRAPS
# ----------------------------------------------------

def average_numerador_denom(df_bootstrap_list, df_transado, df_full, fecha_hora_i_t, gamma, firma):
    """
    Promedia el numerador y denominador kernelizados sobre varias muestras bootstrap.
    Devuelve el promedio (numer, denom) y el último p*, q* observados.
    """
    if not df_bootstrap_list:
        return np.nan, np.nan, None, None

    muestras=len(df_bootstrap_list) 
    
    numeradores = []
    denominadores = []

    # Precomputar para cada bootstrap (vectorizado en el sentido de loops limpios)
    for df_sim in df_bootstrap_list:
        
        df_firma=df_full[
            (df_full['CodigoPlanta'] == firma) &
            (df_full['FechaHora'] == fecha_hora_i_t)]
        
        #todos lo bidders incluyendo i
        df_bidders = pd.concat([df_sim, df_firma], ignore_index=True)
        p_ht_bs,  q_ht_bs = compute_equilibrium(df_bidders, df_transado)
        D= df_transado[(df_transado['FechaHora'] == fecha_hora_i_t)]['demanda']
        
        #exluir a bidder i
        # Numerador ≈ E[-it][Q | s, p = b_it]
        numer = kernel_expectation(df_sim, p_ht_bs, D, gamma, df_firma)

        # Denominador ≈ E[-it][dQ/db | s, p = b_it]
        denom = kernel_derivative_weighted(df_sim, p_ht_bs, gamma, df_firma)    #kernel_derivative
        
        if denom!=0:
            numeradores.append(numer)
            denominadores.append(denom)

    # Convertir a vectores numpy
    numeradores = np.array(numeradores)
    denominadores = np.array(denominadores)

    # Promedios
    avg_numer = numeradores.mean() if len(numeradores) > 0 else 0.0
    avg_denom = denominadores.mean() if len(denominadores) > 0 else 0.0

    return avg_numer, avg_denom, numeradores, denominadores, p_ht_bs,  q_ht_bs
   


In [8]:

# ----------------------------------------------------
# FUNCIÓN PRINCIPAL
# ----------------------------------------------------

# Función auxiliar para el trabajo de una sola fila
def process_row(row, df_full, df_transado, gamma, M):
    fecha = row.FechaHora          # <-- antes row['FechaHora']
    firma = row.CodigoPlanta       # <-- antes row['CodigoPlanta']
    
    # 1) Días similares
    df_similares = get_similar_days_by_cluster(df_full, fecha, firma)
    
    
    # 2) Bootstraps
    df_bootstrap_list = bootstrap_by_planta(df_similares, M, seed=123)

    # 3) RD(b_it) y RD'(b_it) promedio
    avg_numer, avg_denom, numeradores, denominadores, p_ht_bs,  q_ht_bs = average_numerador_denom(
        df_bootstrap_list,
        df_transado,
        df_full,
        fecha,
        gamma,
        firma
    )

    return avg_numer, avg_denom


def calcular_avg_Q_y_dQdb_parallel(df, df_transado, gamma, M, seed=123, n_jobs=-1):

    with tqdm_joblib(tqdm(total=len(df), desc="Calculando", unit="fila")):
        results = Parallel(n_jobs=n_jobs, backend='loky')(
            delayed(process_row)(row, df,df_transado, gamma, M) 
            for row in df.itertuples(index=False)   
        )

    # Separar resultados
    EQ_results = [res[0] for res in results]
    dQdb_results = [res[1] for res in results]

    # Agregar al DataFrame
    df['EQpos'] = EQ_results
    df['EdQb'] = dQdb_results
    
    return df

## Test

In [9]:
def process_row_prueba(row, df_full,  df_transado, gamma, M):
    fecha = row.FechaHora          # <-- antes row['FechaHora']
    firma = row.CodigoPlanta       # <-- antes row['CodigoPlanta']
    
    # 1) Días similares
    df_similares = get_similar_days_by_cluster(df_full, fecha, firma)
    
    
    # 2) Bootstraps
    df_bootstrap_list = bootstrap_by_planta(df_similares, M, seed=123)

    # 3) RD(b_it) y RD'(b_it) promedio
    avg_numer, avg_denom, numeradores, denominadores, p_ht_bs,  q_ht_bs = average_numerador_denom(
        df_bootstrap_list,
        df_transado,
        df_full,
        fecha,
        gamma,
        firma
    )
    
    return df_similares, df_bootstrap_list, avg_numer, avg_denom, p_ht_bs,  q_ht_bs #solo para el test

In [10]:
df_final['FechaHora'] = pd.to_datetime(df_final['FechaHora'], errors='coerce')
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], errors='coerce')
df_transado2=df_transado[df_transado['FechaHora']=='2025-05-25 23:00:00']
df_transado2.loc[1319, 'demanda'] = 180000

row=df_final.iloc[0]
df_full = df_final[df_final['CodigoPlanta'].isin(['JAGS', 'ZPA2', 'TEC1', 'GVIO'])]



df_similares, df_bootstrap_list, avg_numer, avg_denom, p_ht_bs,  q_ht_bs= process_row_prueba(row, df_full, df_transado2, gamma, M)

C:\Users\HP\AppData\Local\Temp\ipykernel_49848\2884614128.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transado_date['FechaHora'] = pd.to_datetime(df_transado_date['FechaHora'], errors='coerce')
C:\Users\HP\AppData\Local\Temp\ipykernel_49848\2884614128.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transado_date['FechaHora'] = pd.to_datetime(df_transado_date['FechaHora'], errors='coerce')
C:\Users\HP\AppData\Local\Temp\ipykernel_49848\2884614128.py:87: SettingWithCopyWarning: 
A value is

KeyboardInterrupt: 

In [ ]:
df_transado2

In [ ]:
df_bootstrap_list[0]

In [ ]:
p_ht_bs

In [ ]:
q_ht_bs

In [ ]:
avg_numer

In [ ]:
avg_denom

In [ ]:
row

## Aplicar a toda la base

In [10]:
######################
### ELIMINAR #########
######################
df_final['FechaHora'] = pd.to_datetime(df_final['FechaHora'], errors='coerce')
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], errors='coerce')

# Filtrar solo filas del 25 de mayo de cualquier año o específico 2025
df_filtrado = df_final[
    (df_final['FechaHora'].dt.year == 2025) &
    (df_final['FechaHora'].dt.month == 5) &
    (df_final['FechaHora'].dt.day >= 1) &
    (df_final['FechaHora'].dt.day <= 15)
]


df_filtrado = df_filtrado.sort_values(
    by=['CodigoPlanta', 'FechaHora'], 
    ascending=[True, True]
).reset_index(drop=True)


In [11]:
df_filtrado.dtypes

FechaHora       datetime64[ns]
CodigoPlanta            object
precio                 float64
cantidad               float64
cantidad_pos           float64
Fecha           datetime64[ns]
cluster                  int64
gamma_thumb            float64
dtype: object

In [12]:
df_resultado = calcular_avg_Q_y_dQdb_parallel(
    df=df_final, 
    df_transado=df_transado,
    gamma=gamma, 
    M=M, 
    n_jobs=-1
)

Calculando:   0%|          | 0/176194 [00:00<?, ?fila/s]

  0%|          | 0/176194 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df_resultado

In [ ]:
 df_resultado['EQpos'] / df_resultado['EdQb']

In [ ]:
df_resultado[df_resultado["EdQb"] > 0]


In [ ]:
df_resultado[df_resultado["precio"] < 0]

## Guardar resultados 

In [ ]:

# Crear carpeta principal
output_dir = "../results"
os.makedirs(output_dir, exist_ok=True)
print(f"Carpeta creada: {output_dir}")

In [ ]:

# Calcular número de fechas únicas
n_fechas = df_resultadol["Fecha"].nunique()

# Crear subcarpeta con el número de fechas únicas
sub_dir = os.path.join(output_dir, f"fechas_{n_fechas}").replace("\\", "/")
os.makedirs(sub_dir, exist_ok=True)

print(f"Carpeta creada: {sub_dir}")


In [ ]:
# --- Guardar CSV ---
# Convertimos las fechas al formato YYYYMMDD (sin caracteres prohibidos)
fecha_min_str = pd.to_datetime(df_resultadol['Fecha'].min()).strftime('%Y%m%d')
fecha_max_str = pd.to_datetime(df_resultadol['Fecha'].max()).strftime('%Y%m%d')

filename = f"df_{fecha_min_str}_to_{fecha_max_str}_M{M}_preproc.csv"

# Forzar que la ruta use '/' en vez de '\'
filepath = os.path.join(sub_dir, filename).replace("\\", "/")

# Crear carpeta si no existe
os.makedirs(sub_dir, exist_ok=True)

#df_resultado.to_csv(filepath, index=False)
print(f"Archivo guardado en: {filepath}")


In [ ]:
df_resultado

In [ ]:
df_resultado['EQpos'].describe()